In [45]:
# Run reviews_exploratiion.ipynb first.
import pandas as pd
import json
import ast
import numpy as np
import math
import missingno as msno

In [46]:
# Load the reviews data.
games_reviews_long_df = pd.read_csv('games_reviews_flat.csv')
games_reviews_long_df.head()

,Unnamed: 0,igdb_id,first_release_date,name,slug,mature,single_player,multiplayer,dev1_stock_id,dev2_stock_id,...,topCriticScore,percentile,firstReleaseDate,firstReviewDate,latestReviewDate,variable,igdb_company_id,developer,publisher,stock_id
0,2,117,1480982400,The Last Guardian,the-last-guardian,False,True,False,0,0,...,80.528736,83.0,2016-12-06T00:00:00.000Z,2016-12-05T00:00:00.000Z,2017-09-10T04:00:00.000Z,pub1_stock_id,10100,False,True,JP3435000009
1,4,359,1480377600,Final Fantasy XV,final-fantasy-xv,False,True,False,26,0,...,80.736842,83.0,2016-11-29T00:00:00.000Z,2016-08-15T04:00:00.000Z,2018-03-08T05:00:00.000Z,dev1_stock_id,26,True,False,JP3164630000
2,6,359,1480377600,Final Fantasy XV,final-fantasy-xv,False,True,False,26,0,...,80.736842,83.0,2016-11-29T00:00:00.000Z,2016-08-15T04:00:00.000Z,2018-03-08T05:00:00.000Z,pub1_stock_id,26,False,True,JP3164630000
3,10,2112,1465257600,Mirror's Edge Catalyst,mirrors-edge-catalyst,False,True,False,0,0,...,70.168831,44.0,2016-06-07T00:00:00.000Z,2016-04-26T04:00:00.000Z,2017-08-14T00:00:00.000Z,pub1_stock_id,1,False,True,16721
4,14,2212,1459209600,MLB The Show 16,mlb-the-show-16,False,True,False,0,0,...,85.464286,95.0,2016-03-29T00:00:00.000Z,2016-03-28T04:00:00.000Z,2016-06-15T04:00:00.000Z,pub1_stock_id,13634,False,True,JP3435000009


In [47]:
games_reviews_long_df.columns

Index(['Unnamed: 0', 'igdb_id', 'first_release_date', 'name', 'slug', 'mature',
       'single_player', 'multiplayer', 'dev1_stock_id', 'dev2_stock_id',
       'pub1_stock_id', 'pub2_stock_id', 'igdb_name', 'opencritic_name',
       'opencritic_id', 'dist', 'hasLootBoxes', 'percentRecommended',
       'numReviews', 'numTopCriticReviews', 'medianScore', 'topCriticScore',
       'percentile', 'firstReleaseDate', 'firstReviewDate', 'latestReviewDate',
       'variable', 'igdb_company_id', 'developer', 'publisher', 'stock_id'],
      dtype='object')

In [48]:
# Parse first release date as datetime.
games_reviews_long_df['first_release_date'] = pd.to_datetime(games_reviews_long_df['first_release_date'],unit='s')
games_reviews_long_df.dtypes

Unnamed: 0                      int64
igdb_id                         int64
first_release_date     datetime64[ns]
name                           object
slug                           object
mature                           bool
single_player                    bool
multiplayer                      bool
dev1_stock_id                   int64
dev2_stock_id                   int64
pub1_stock_id                   int64
pub2_stock_id                   int64
igdb_name                      object
opencritic_name                object
opencritic_id                   int64
dist                          float64
hasLootBoxes                   object
percentRecommended            float64
numReviews                    float64
numTopCriticReviews           float64
medianScore                   float64
topCriticScore                float64
percentile                    float64
firstReleaseDate               object
firstReviewDate                object
latestReviewDate               object
variable    

In [49]:
# Create date column from first_release_date for joining
games_reviews_long_df['date'] = games_reviews_long_df['first_release_date']

In [50]:
games_reviews_long_df.head()

,Unnamed: 0,igdb_id,first_release_date,name,slug,mature,single_player,multiplayer,dev1_stock_id,dev2_stock_id,...,percentile,firstReleaseDate,firstReviewDate,latestReviewDate,variable,igdb_company_id,developer,publisher,stock_id,date
0,2,117,2016-12-06,The Last Guardian,the-last-guardian,False,True,False,0,0,...,83.0,2016-12-06T00:00:00.000Z,2016-12-05T00:00:00.000Z,2017-09-10T04:00:00.000Z,pub1_stock_id,10100,False,True,JP3435000009,2016-12-06
1,4,359,2016-11-29,Final Fantasy XV,final-fantasy-xv,False,True,False,26,0,...,83.0,2016-11-29T00:00:00.000Z,2016-08-15T04:00:00.000Z,2018-03-08T05:00:00.000Z,dev1_stock_id,26,True,False,JP3164630000,2016-11-29
2,6,359,2016-11-29,Final Fantasy XV,final-fantasy-xv,False,True,False,26,0,...,83.0,2016-11-29T00:00:00.000Z,2016-08-15T04:00:00.000Z,2018-03-08T05:00:00.000Z,pub1_stock_id,26,False,True,JP3164630000,2016-11-29
3,10,2112,2016-06-07,Mirror's Edge Catalyst,mirrors-edge-catalyst,False,True,False,0,0,...,44.0,2016-06-07T00:00:00.000Z,2016-04-26T04:00:00.000Z,2017-08-14T00:00:00.000Z,pub1_stock_id,1,False,True,16721,2016-06-07
4,14,2212,2016-03-29,MLB The Show 16,mlb-the-show-16,False,True,False,0,0,...,95.0,2016-03-29T00:00:00.000Z,2016-03-28T04:00:00.000Z,2016-06-15T04:00:00.000Z,pub1_stock_id,13634,False,True,JP3435000009,2016-03-29


In [51]:
# Load the financial data.
stock_ff_index_df = pd.read_csv('financial_data_clean.csv')
stock_ff_index_df.head()

,Unnamed: 0,gvkey,iid,date,company_name,shares_outstanding,trading_volume,closing_price,total_return_factor,stock_id,...,mkt-rf,smb,hml,rmw,cma,rf,benchmark_price,returns,benchmark_returns,excess_return
0,0,212596,01W,2015-10-01,UBI SOFT ENTERTAINMENT SA,111218465.0,506501.0,18.420,1.008395,FR0000054470,...,-0.29,0.52,0.24,-0.11,0.22,0.0,308.80,NaN,NaN,NaN
1,1,212596,01W,2015-10-02,UBI SOFT ENTERTAINMENT SA,111218465.0,217260.0,18.155,1.008395,FR0000054470,...,0.93,-0.36,0.14,-0.25,0.22,0.0,310.56,-0.014387,0.005699,-0.020086
2,2,212596,01W,2015-10-05,UBI SOFT ENTERTAINMENT SA,111218465.0,267647.0,18.355,1.008395,FR0000054470,...,2.37,-1.31,0.43,-0.20,0.06,0.0,319.53,0.011016,0.028883,-0.017867
3,3,212596,01W,2015-10-06,UBI SOFT ENTERTAINMENT SA,111218465.0,984676.0,19.900,1.008395,FR0000054470,...,1.33,-0.10,0.66,0.04,0.05,0.0,321.06,0.084173,0.004788,0.079385
4,4,212596,01W,2015-10-07,UBI SOFT ENTERTAINMENT SA,111218465.0,740257.0,20.120,1.008395,FR0000054470,...,-0.07,0.07,1.52,-0.54,0.56,0.0,319.89,0.011055,-0.003644,0.014699


In [52]:
stock_ff_index_df.columns

Index(['Unnamed: 0', 'gvkey', 'iid', 'date', 'company_name',
       'shares_outstanding', 'trading_volume', 'closing_price',
       'total_return_factor', 'stock_id', 'eu', 'jp', 'us', 'mkt-rf', 'smb',
       'hml', 'rmw', 'cma', 'rf', 'benchmark_price', 'returns',
       'benchmark_returns', 'excess_return'],
      dtype='object')

In [53]:
stock_ff_index_df.dtypes

Unnamed: 0               int64
gvkey                    int64
iid                     object
date                    object
company_name            object
shares_outstanding     float64
trading_volume         float64
closing_price          float64
total_return_factor    float64
stock_id                object
eu                        bool
jp                        bool
us                        bool
mkt-rf                 float64
smb                    float64
hml                    float64
rmw                    float64
cma                    float64
rf                     float64
benchmark_price        float64
returns                float64
benchmark_returns      float64
excess_return          float64
dtype: object

In [54]:
# Convert stock_data_df date column to datetime.
stock_ff_index_df['date'] = pd.to_datetime(stock_ff_index_df['date'])
stock_ff_index_df.head()

,Unnamed: 0,gvkey,iid,date,company_name,shares_outstanding,trading_volume,closing_price,total_return_factor,stock_id,...,mkt-rf,smb,hml,rmw,cma,rf,benchmark_price,returns,benchmark_returns,excess_return
0,0,212596,01W,2015-10-01,UBI SOFT ENTERTAINMENT SA,111218465.0,506501.0,18.420,1.008395,FR0000054470,...,-0.29,0.52,0.24,-0.11,0.22,0.0,308.80,NaN,NaN,NaN
1,1,212596,01W,2015-10-02,UBI SOFT ENTERTAINMENT SA,111218465.0,217260.0,18.155,1.008395,FR0000054470,...,0.93,-0.36,0.14,-0.25,0.22,0.0,310.56,-0.014387,0.005699,-0.020086
2,2,212596,01W,2015-10-05,UBI SOFT ENTERTAINMENT SA,111218465.0,267647.0,18.355,1.008395,FR0000054470,...,2.37,-1.31,0.43,-0.20,0.06,0.0,319.53,0.011016,0.028883,-0.017867
3,3,212596,01W,2015-10-06,UBI SOFT ENTERTAINMENT SA,111218465.0,984676.0,19.900,1.008395,FR0000054470,...,1.33,-0.10,0.66,0.04,0.05,0.0,321.06,0.084173,0.004788,0.079385
4,4,212596,01W,2015-10-07,UBI SOFT ENTERTAINMENT SA,111218465.0,740257.0,20.120,1.008395,FR0000054470,...,-0.07,0.07,1.52,-0.54,0.56,0.0,319.89,0.011055,-0.003644,0.014699


Begin by joining the reviews dataset with the financial dataset on release date and stock_id.

In [55]:
stock_games_df = pd.merge(stock_ff_index_df, games_reviews_long_df, how='left', on=['date', 'stock_id'])
stock_games_df.head()

,Unnamed: 0_x,gvkey,iid,date,company_name,shares_outstanding,trading_volume,closing_price,total_return_factor,stock_id,...,medianScore,topCriticScore,percentile,firstReleaseDate,firstReviewDate,latestReviewDate,variable,igdb_company_id,developer,publisher
0,0,212596,01W,2015-10-01,UBI SOFT ENTERTAINMENT SA,111218465.0,506501.0,18.420,1.008395,FR0000054470,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,212596,01W,2015-10-02,UBI SOFT ENTERTAINMENT SA,111218465.0,217260.0,18.155,1.008395,FR0000054470,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,212596,01W,2015-10-05,UBI SOFT ENTERTAINMENT SA,111218465.0,267647.0,18.355,1.008395,FR0000054470,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,212596,01W,2015-10-06,UBI SOFT ENTERTAINMENT SA,111218465.0,984676.0,19.900,1.008395,FR0000054470,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,212596,01W,2015-10-07,UBI SOFT ENTERTAINMENT SA,111218465.0,740257.0,20.120,1.008395,FR0000054470,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
